In [5]:
import pandas as pd

In [12]:
pip install xlrd

     ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
     ------------------------ ------------- 61.4/96.5 kB 544.7 kB/s eta 0:00:01
     -------------------------------------- 96.5/96.5 kB 612.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
shark_data = pd.read_excel(url)

In [16]:
shark_data.dtypes

Date               object
Year              float64
Type               object
Country            object
State              object
Location           object
Activity           object
Name               object
Sex                object
Age                object
Injury             object
Unnamed: 11        object
Time               object
Species            object
Source             object
pdf                object
href formula       object
href               object
Case Number        object
Case Number.1      object
original order    float64
Unnamed: 21        object
Unnamed: 22        object
dtype: object

In [17]:
shark_data.nunique()

Date              5992
Year               259
Type                11
Country            224
State              896
Location          4503
Activity          1586
Name              5677
Sex                  8
Age                243
Injury            4076
Unnamed: 11         12
Time               410
Species           1674
Source            5291
pdf               6789
href formula      6785
href              6776
Case Number       6777
Case Number.1     6775
original order    6797
Unnamed: 21          1
Unnamed: 22          2
dtype: int64